In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
empty = pd.read_csv('/content/drive/MyDrive/미거주주택(빈집)현황(구별).csv',header=None)

In [ ]:
# 두 번째 행(1번 인덱스)을 컬럼명으로 설정
empty.columns = empty.iloc[2].values
empty = empty.set_index('자치구별(1)')

In [ ]:
empty = empty.drop(empty.index[0:2])

In [ ]:
empty.head()

,소계,단독주택,아파트,연립주택,다세대주택,비주거용 건물내 주택
자치구별(1),,,,,,
계,102556,4603,50820,5890,39546,1697
종로구,2123,298,643,308,760,114
중구,1766,58,1334,37,216,121
용산구,7499,1553,2936,617,2281,112
성동구,1957,23,1338,88,429,79


In [ ]:
house = pd.read_csv('/content/drive/MyDrive/주택수.csv')

In [ ]:
house = house.drop(house.index[0:1])   # 첫 행 제거
house = house.rename(columns={'동별(2)': '자치구', '2024': '전체주택수'})
house = house.set_index('자치구')

In [ ]:
house.head()

,동별(1),전체주택수
자치구,,
소계,서울시,3170332
종로구,서울시,47156
중구,서울시,44572
용산구,서울시,78063
성동구,서울시,91231


In [ ]:
old = pd.read_csv('/content/drive/MyDrive/노후기간별주택현황.csv')
old = old.loc[2:,['자치구(2)','2024.6']]
old = old.rename(columns={'자치구(2)': '자치구', '2024.6': '노후주택수'})
old.set_index('자치구', inplace=True)
old.head()

,노후주택수
자치구,
소계,897159
종로구,16715
중구,9793
용산구,25977
성동구,20927


In [ ]:
# 인덱스가 자치구로 되어있는 경우를 대비해 모두 해제
for df in [empty, house, old]:
    if df.index.name == '자치구' or '자치구' not in df.columns:
        df.reset_index(inplace=True)

# 컬럼 이름 확인 (자치구가 컬럼으로 존재하는지)
print(empty.columns)
print(house.columns)
print(old.columns)




Index(['자치구별(1)', '소계', '단독주택', '아파트', '연립주택', '다세대주택', '비주거용 건물내 주택'], dtype='object')
Index(['자치구', '동별(1)', '전체주택수'], dtype='object')
Index(['자치구', '노후주택수'], dtype='object')


In [ ]:
# 컬럼명 통일
empty = empty.rename(columns={'자치구별(1)': '자치구'})

# 병합
merged = (
    empty
    .merge(house[['자치구','전체주택수']], on='자치구', how='left')
    .merge(old[['자치구','노후주택수']], on='자치구', how='left')
)

In [ ]:
# 숫자형 변환
for col in ['소계','단독주택','연립주택','다세대주택','아파트','비주거용 건물내 주택','전체주택수','노후주택수']:
    merged[col] = merged[col].astype(str).str.replace(',','').astype(float)

# 파생변수 계산
merged['거주용빈집수'] = merged[['단독주택','연립주택','다세대주택','아파트']].sum(axis=1)
merged['비거주용빈집수'] = merged['비주거용 건물내 주택']
merged['전체빈집비율'] = merged['소계'] / merged['전체주택수']
merged['거주용빈집비율'] = merged['거주용빈집수'] / merged['전체주택수']
merged['비거주용빈집비율'] = merged['비거주용빈집수'] / merged['전체주택수']
merged['노후주택비율'] = merged['노후주택수'] / merged['전체주택수']

merged.head()


,자치구,소계,단독주택,아파트,연립주택,다세대주택,비주거용 건물내 주택,전체주택수,노후주택수,거주용빈집수,비거주용빈집수,전체빈집비율,거주용빈집비율,비거주용빈집비율,노후주택비율
0,계,102556.0,4603.0,50820.0,5890.0,39546.0,1697.0,NaN,NaN,100859.0,1697.0,NaN,NaN,NaN,NaN
1,종로구,2123.0,298.0,643.0,308.0,760.0,114.0,47156.0,16715.0,2009.0,114.0,0.045021,0.042603,0.002418,0.354462
2,중구,1766.0,58.0,1334.0,37.0,216.0,121.0,44572.0,9793.0,1645.0,121.0,0.039621,0.036907,0.002715,0.219712
3,용산구,7499.0,1553.0,2936.0,617.0,2281.0,112.0,78063.0,25977.0,7387.0,112.0,0.096063,0.094629,0.001435,0.332770
4,성동구,1957.0,23.0,1338.0,88.0,429.0,79.0,91231.0,20927.0,1878.0,79.0,0.021451,0.020585,0.000866,0.229385


In [ ]:
merged = merged.drop(0,axis=0)

## 노후주택비율

In [ ]:
old = merged[['자치구','노후주택비율']]
old.head()

,자치구,노후주택비율
1,종로구,0.354462
2,중구,0.219712
3,용산구,0.332770
4,성동구,0.229385
5,광진구,0.257333


## 아파트비율

In [ ]:
# '아파트 비율' 변수 생성
empty['apartrate'] = empty['아파트'].astype(float) / empty['소계'].astype(float)

print(empty[['소계','아파트','apartrate']].head())

       소계    아파트  apartrate
0  102556  50820   0.495534
1    2123    643   0.302873
2    1766   1334   0.755379
3    7499   2936   0.391519
4    1957   1338   0.683700


## 순이동률

In [ ]:
migration = pd.read_csv('/content/drive/MyDrive/인구이동률(구별).csv')
migration = migration.loc[4:,['자치구별(1)','2024.21']]
# 열 이름 변경
migration = migration.rename(columns={'자치구별(1)': '자치구', '2024.21': '순이동률'})


# 확인
print(migration.head())

   자치구  순이동률
4  서울시  -0.5
5  종로구  -0.5
6   중구  -0.4
7  용산구  -4.3
8  성동구  -1.4


## 고령인구비율

In [ ]:
senior = pd.read_csv('/content/drive/MyDrive/고령자현황.csv')

In [ ]:
senior = senior.drop('동별(1)',axis=1)
senior = senior.drop([0,1],axis=0)
senior = senior.rename(columns={'동별(2)':'자치구','2025 2/4':'65세 인구'})


In [ ]:
people = pd.read_csv('/content/drive/MyDrive/등록인구.csv')
people = people.drop('동별(1)',axis=1)
people = people.drop(0,axis=0)
people = people.rename(columns={'동별(2)':'자치구','2025 2/4':'인구'})

In [ ]:
# 숫자형 변환 (쉼표 제거 후 float 변환)
people["인구"] = people["인구"].astype(str).str.replace(",", "").astype(float)
senior["65세 인구"] = senior["65세 인구"].astype(str).str.replace(",", "").astype(float)

# 1. 두 데이터 병합 (자치구 기준)
seniorrate = pd.merge(people, senior, on="자치구", how="inner")

# 2. 고령인구 비율 계산
seniorrate["고령인구비율"] = seniorrate["65세 인구"] / seniorrate["인구"]

# 3. 확인
print(seniorrate.head())

   자치구         인구     65세 인구    고령인구비율
0   소계  9325616.0  1860632.0  0.199518
1  종로구   138015.0    30109.0  0.218157
2   중구   119097.0    26805.0  0.225069
3  용산구   203127.0    39496.0  0.194440
4  성동구   273195.0    52186.0  0.191021


### 인구천명당_의료병상수(의료접근성)

In [ ]:
hospital = pd.read_csv('/content/drive/MyDrive/의료기관(구별).csv')
hospital = hospital.drop('자치구별(1)',axis=1)
hospital = hospital.drop([0,1],axis=0)
hospital = hospital.rename(columns={'자치구별(2)':'자치구','2024':'병상수'})
hospital.head()

,자치구,병상수
2,소계,88458
3,종로구,3527
4,중구,931
5,용산구,810
6,성동구,1771


In [ ]:
# 숫자형 변환 (쉼표 제거 후 float 변환)
people["인구"] = people["인구"].astype(str).str.replace(",", "").astype(float)
hospital["병상수"] = hospital["병상수"].astype(str).str.replace(",", "").astype(float)

hos1000 = pd.merge(hospital, people, on="자치구", how="inner")

# 천명당 병상수 계산
hos1000["천명당_병상수"] = hos1000["병상수"] / hos1000["인구"] * 1000

print(hos1000.head())

   자치구      병상수         인구    천명당_병상수
0   소계  88458.0  9325616.0   9.485486
1  종로구   3527.0   138015.0  25.555193
2   중구    931.0   119097.0   7.817157
3  용산구    810.0   203127.0   3.987653
4  성동구   1771.0   273195.0   6.482549


## 교육접근성 (교육인프라/전체인구)*10000

In [ ]:
edu = pd.read_csv('/content/drive/MyDrive/학교총괄.csv')
edu = edu.drop(0,axis=0)
edu = edu.drop('학교급별(1)',axis=1)
edu.columns = ['소계','2024']
edu.head()

,소계,2024
1,소계,2606
2,유치원,754
3,초등학교,609
4,중학교,390
5,일반고등학교,212


## 경제활동인구

In [ ]:
economy = pd.read_csv('/content/drive/MyDrive/경제활동인구.csv')
economy = economy.drop('항목',axis=1)
economy = economy.drop(0,axis=0)
economy = economy.rename(columns={'동별(1)':'자치구'})

# ✅ 숫자형으로 변환 (자치구 제외한 모든 열)
for col in economy.columns[1:]:
    economy[col] = pd.to_numeric(economy[col], errors='coerce')

# '자치구'를 제외한 나머지 열들의 합계를 계산
economy['경제활동인구'] = economy.iloc[:, 1:].sum(axis=1)

# '자치구'와 합계열만 남기기
economy = economy[['자치구', '경제활동인구']]

# 확인
economy.head()

,자치구,경제활동인구
1,종로구,108863
2,중구,94603
3,용산구,158642
4,성동구,204952
5,광진구,262389


## 인구밀도

In [ ]:
density = pd.read_csv('/content/drive/MyDrive/인구밀도.csv')
density = density.drop([0,1],axis=0)
density = density.drop(['동별(1)','동별(3)','2024','2024.1'],axis=1)
density = density.rename(columns={'동별(2)':'자치구','2024.2':'인구밀도'})
density.head()


,자치구,인구밀도
2,종로구,6256
3,중구,13174
4,용산구,9932
5,성동구,16723
6,광진구,20433


## 자동차등록수

In [ ]:
car = pd.read_csv('/content/drive/MyDrive/자동차등록(월별_구별).csv')
car = car.drop('자치구별(1)',axis=1)
car = car.drop([0,1,2,3],axis=0)
car = car.drop('2024. 12',axis=1)
car = car.rename(columns={'자치구별(2)':'자치구','2024. 12.1':'자동차등록수'})
car.head()

,자치구,자동차등록수
4,종로구,39262
5,중구,40747
6,용산구,63515
7,성동구,82022
8,광진구,80086


## 주택가격지수

In [ ]:
price = pd.read_csv("/content/drive/MyDrive/주택가격지수(매매).csv")
price = price.iloc[2:].reset_index(drop=True)
price = price.rename(columns={"자치구별(2)":"자치구"})
price["주택가격지수"] = pd.to_numeric(price["2024. 12"], errors="coerce")
price = price[["자치구","주택가격지수"]]
price.head()

,자치구,주택가격지수
0,중구,97.4
1,용산구,106.4
2,성동구,106.6
3,광진구,100.7
4,동대문구,98.6


## 자살률

In [ ]:
suicide = pd.read_csv("/content/drive/MyDrive/자살률(구별).csv")
suicide = suicide.iloc[2:].reset_index(drop=True)
suicide = suicide.rename(columns={"자치구별(2)":"자치구"})
suicide["자살률"] = pd.to_numeric(suicide["2024.3"], errors="coerce")  # 소계 자살률
suicide = suicide[["자치구","자살률"]]
suicide = suicide.drop(0,axis=0)
suicide.head()

,자치구,자살률
1,소계,24.1
2,종로구,29.2
3,중구,26.8
4,용산구,21.0
5,성동구,27.1


## 교육환경만족도

In [ ]:
edu = pd.read_csv("/content/drive/MyDrive/교육환경만족도.csv")
edu = edu.drop(0,axis=0)
edu = edu.drop('구분별(1)',axis=1)
edu = edu.rename(columns={"구분별(2)":"자치구","2024":"교육만족도"})
edu.head()

,자치구,교육만족도
1,종로구,5.53
2,중구,6.76
3,용산구,6.19
4,성동구,6.36
5,광진구,5.17


## 문화공간

In [ ]:
cultural = pd.read_csv("/content/drive/MyDrive/문화공간.csv")
cultural = cultural.drop('자치구별(1)',axis=1)
cultural = cultural.drop([0,1],axis=0)
cultural['2024'] = pd.to_numeric(cultural['2024'], errors='coerce')
cultural['2024.1'] = pd.to_numeric(cultural['2024.1'], errors='coerce')
cultural[['2024', '2024.1']] = cultural[['2024', '2024.1']].fillna(0)
cultural['문화공간'] = cultural['2024'] + cultural['2024.1']
cultural = cultural.drop(['2024', '2024.1'], axis=1)
cultural = cultural.rename(columns={"자치구별(2)":"자치구"})
cultural.head()

,자치구,문화공간
2,종로구,57.0
3,중구,20.0
4,용산구,12.0
5,성동구,5.0
6,광진구,3.0


## 노인복지시설

In [ ]:
welfare = pd.read_csv("/content/drive/MyDrive/노인여가복지시설(구별).csv")
welfare = welfare.iloc[2:].reset_index(drop=True)
welfare = welfare.rename(columns={"자치구(2)":"자치구"})
welfare["노인복지시설수"] = pd.to_numeric(welfare["2023"], errors="coerce")  # 시설합계
welfare = welfare[["자치구","노인복지시설수"]]
welfare.head()

,자치구,노인복지시설수
0,소계,4051
1,종로구,77
2,중구,55
3,용산구,114
4,성동구,183


## 1인가구

In [ ]:
one = pd.read_csv("/content/drive/MyDrive/1인가구(연령별).csv")
one = one.iloc[2:].reset_index(drop=True)
one = one.rename(columns={"자치구별(2)":"자치구"})
one["1인가구수"] = pd.to_numeric(one["2024"], errors="coerce")
one = one[["자치구","1인가구수"]]
one.head()

,자치구,1인가구수
0,종로구,29779
1,중구,28968
2,용산구,40445
3,성동구,48442
4,광진구,72933


In [ ]:
# 1️⃣ 먼저 첫 번째 데이터프레임부터 시작
df = old.copy()

# 2️⃣ 자치구 기준으로 순차 병합
df = df.merge(merged[['자치구','전체빈집비율','거주용빈집비율','비거주용빈집비율']], on="자치구", how="inner")
df = df.merge(seniorrate[['자치구','고령인구비율']], on="자치구", how="inner")
df = df.merge(density, on="자치구", how="inner")
df = df.merge(migration, on="자치구", how="inner")
df = df.merge(economy, on="자치구", how="inner")
df = df.merge(hos1000[['자치구','천명당_병상수']], on="자치구", how="inner")
df = df.merge(edu[['자치구','교육만족도']], on="자치구", how="inner")
df = df.merge(cultural[['자치구','문화공간']], on="자치구", how="inner")
df = df.merge(car[['자치구','자동차등록수']], on="자치구", how="inner")
df = df.merge(price[['자치구','주택가격지수']], on="자치구", how="inner")
df = df.merge(suicide[['자치구','자살률']], on="자치구", how="inner")
df = df.merge(welfare[['자치구','노인복지시설수']], on="자치구", how="inner")
df = df.merge(one[['자치구','1인가구수']], on="자치구", how="inner")

# 3️⃣ 병합 결과 확인
print(df.head())
print(df.shape)


    자치구    노후주택비율    전체빈집비율   거주용빈집비율  비거주용빈집비율    고령인구비율   인구밀도  순이동률  \
0    중구  0.219712  0.039621  0.036907  0.002715  0.225069  13174  -0.4   
1   용산구  0.332770  0.096063  0.094629  0.001435  0.194440   9932  -4.3   
2   성동구  0.229385  0.021451  0.020585  0.000866  0.191021  16723  -1.4   
3   광진구  0.257333  0.011157  0.010854  0.000303  0.183530  20433  -1.0   
4  동대문구  0.222163  0.021388  0.020603  0.000785  0.206798  25226  -0.5   

   경제활동인구    천명당_병상수 교육만족도  문화공간 자동차등록수  주택가격지수   자살률  노인복지시설수  1인가구수  
0   94603   7.817157  6.76  20.0  40747    97.4  26.8       55  28968  
1  158642   3.987653  6.19  12.0  63515   106.4  21.0      114  40445  
2  204952   6.482549  6.36   5.0  82022   106.6  27.1      183  48442  
3  262389   7.493335  5.17   3.0  80086   100.7  27.5      107  72933  
4  262912  15.396344  5.31   5.0  84500    98.6  23.4      155  72355  
(24, 17)


## 크루스칼-왈리스 검정

In [ ]:
import scipy.stats as stats


# 분석 변수 리스트 (빈집수 제외)
variables = [
    "노후주택비율","고령인구비율","인구밀도","순이동률",
    "경제활동인구","천명당_병상수","교육만족도","문화공간","자동차등록수","주택가격지수","자살률","노인복지시설수","1인가구수"
]

# 모든 변수 숫자형 변환
for var in variables:
    df[var] = pd.to_numeric(df[var], errors='coerce')

# 권역 그룹 (예시: 서울시 5대 권역)
groups = {
    "동남권": ["강남구","서초구","송파구","강동구"],
    "동북권": ["노원구","도봉구","강북구","중랑구","성북구","동대문구"],
    "서남권": ["관악구","동작구","영등포구","구로구","금천구","강서구","양천구"],
    "서북권": ["은평구","서대문구","마포구"],
    "도심권": ["종로구","중구","용산구"]
}

# -----------------------------
# 각 변수별로 검정 수행
# -----------------------------
for var in variables:
    print(f"\n📌 변수: {var}")

    # ------------------
    # 1. 정규성 검정
    # ------------------
    shapiro_stat, shapiro_p = stats.shapiro(df[var])
    print(f"   Shapiro-Wilk 정규성 p={shapiro_p:.4f}",
          "➡ 정규분포" if shapiro_p>0.05 else "➡ 비정규분포")

    # ------------------
    # 2. 권역별 데이터 추출
    # ------------------
    region_data = [df[df["자치구"].isin(g)][var].dropna() for g in groups.values()]

    # ------------------
    # 3. 등분산 검정 (Levene)
    # ------------------
    if all(len(r) > 1 for r in region_data):
        lev_stat, lev_p = stats.levene(*region_data)
        print(f"   Levene 등분산 p={lev_p:.4f}",
              "➡ 등분산 만족" if lev_p>0.05 else "➡ 등분산 위배")
    else:
        print("   ⚠️ 등분산 검정 불가 (표본 부족)")

    # ------------------
    # 4. 크루스칼-왈리스 검정
    # ------------------
    kruskal_stat, kruskal_p = stats.kruskal(*region_data)
    print(f"   Kruskal-Wallis p={kruskal_p:.4f}",
          "➡ 권역 간 차이 없음" if kruskal_p>0.05 else "➡ 권역 간 차이 있음")



📌 변수: 노후주택비율
   Shapiro-Wilk 정규성 p=0.0020 ➡ 비정규분포
   Levene 등분산 p=0.0169 ➡ 등분산 위배
   Kruskal-Wallis p=0.5072 ➡ 권역 간 차이 없음

📌 변수: 고령인구비율
   Shapiro-Wilk 정규성 p=0.1990 ➡ 정규분포
   Levene 등분산 p=0.3481 ➡ 등분산 만족
   Kruskal-Wallis p=0.0261 ➡ 권역 간 차이 있음

📌 변수: 인구밀도
   Shapiro-Wilk 정규성 p=0.9091 ➡ 정규분포
   Levene 등분산 p=0.3887 ➡ 등분산 만족
   Kruskal-Wallis p=0.2698 ➡ 권역 간 차이 없음

📌 변수: 순이동률
   Shapiro-Wilk 정규성 p=0.0000 ➡ 비정규분포
   Levene 등분산 p=0.0000 ➡ 등분산 위배
   Kruskal-Wallis p=0.1450 ➡ 권역 간 차이 없음

📌 변수: 경제활동인구
   Shapiro-Wilk 정규성 p=0.9932 ➡ 정규분포
   Levene 등분산 p=0.9192 ➡ 등분산 만족
   Kruskal-Wallis p=0.0342 ➡ 권역 간 차이 있음

📌 변수: 천명당_병상수
   Shapiro-Wilk 정규성 p=0.5133 ➡ 정규분포
   Levene 등분산 p=0.7687 ➡ 등분산 만족
   Kruskal-Wallis p=0.1743 ➡ 권역 간 차이 없음

📌 변수: 교육만족도
   Shapiro-Wilk 정규성 p=0.4802 ➡ 정규분포
   Levene 등분산 p=0.0661 ➡ 등분산 만족
   Kruskal-Wallis p=0.2844 ➡ 권역 간 차이 없음

📌 변수: 문화공간
   Shapiro-Wilk 정규성 p=0.0047 ➡ 비정규분포
   Levene 등분산 p=0.4450 ➡ 등분산 만족
   Kruskal-Wallis p=0.0446 ➡ 권역 간 차이 있음

📌 변수: 자동차등록수
   Shapiro-Wi

## 다중공선성

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# 분석에 사용할 변수 선택
X = df[[ "노후주택비율","고령인구비율","인구밀도","순이동률",
    "경제활동인구","천명당_병상수","교육만족도","문화공간","자동차등록수","주택가격지수","자살률","노인복지시설수","1인가구수"]]

# 결측치 처리
X = X.fillna(X.mean())


# 상수항 추가 (VIF 계산에 필요)
X_const = add_constant(X)


# VIF 계산
vif_data = pd.DataFrame()
vif_data["변수"] = X_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_const.values, i)
                   for i in range(X_const.shape[1])]

# 결과 출력
print("📊 다중공선성 분석 결과 (VIF)")
print(vif_data)

📊 다중공선성 분석 결과 (VIF)
         변수          VIF
0     const  5306.047804
1    노후주택비율     1.615733
2    고령인구비율     4.704735
3      인구밀도     2.851496
4      순이동률     2.770766
5    경제활동인구    42.945531
6   천명당_병상수     3.756246
7     교육만족도     2.174964
8      문화공간     2.475911
9    자동차등록수    28.416884
10   주택가격지수     4.752255
11      자살률     2.875849
12  노인복지시설수     3.967961
13    1인가구수     8.718998


## 베타회귀

In [ ]:
!pip install pybetareg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 4.0 MB/s eta 0:00:00


In [ ]:
# ===============================================
# 1️⃣ 라이브러리
# ===============================================
import numpy as np
import pandas as pd
import pybetareg as pyb
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

# ===============================================
# 2️⃣ 데이터 전처리
# ===============================================
df["거주용빈집비율"] = df["거주용빈집비율"].clip(0.001, 0.999)
df["비거주용빈집비율"] = df["비거주용빈집비율"].clip(0.001, 0.999)

features_res = ["노후주택비율", "인구밀도", "순이동률", "1인가구수"]
features_nonres = ["경제활동인구", "문화공간", "주택가격지수", "교육만족도"]

scaler_res = StandardScaler()
scaler_nonres = StandardScaler()

X_res_scaled = scaler_res.fit_transform(df[features_res])
X_nonres_scaled = scaler_nonres.fit_transform(df[features_nonres])

y_res = df["거주용빈집비율"].to_numpy()
y_nonres = df["비거주용빈집비율"].to_numpy()

# ===============================================
# 3️⃣ 베타회귀 적합
# ===============================================
print("🏠 거주용 빈집비율 회귀결과 (logit link)")
model_res = pyb.reg(y=y_res, x=X_res_scaled, link="logit", column_names=features_res)
result_res = model_res.fit()

print("\n🏢 비거주용 빈집비율 회귀결과 (logit link)")
model_nonres = pyb.reg(y=y_nonres, x=X_nonres_scaled, link="logit", column_names=features_nonres)
result_nonres = model_nonres.fit()

# ===============================================
# 4️⃣ 표 형태로 변환 + 출력 함수
# ===============================================
def extract_table(result, title):
    params = result.params
    df = pd.DataFrame({
        "변수명": list(params.keys()),
        "계수(Beta)": [round(v, 4) for v in params.values()]
    })
    print(f"\n📊 {title}")
    display(df)  # Colab, Jupyter에서 표로 바로 보이게 출력
    return df

# ===============================================
# 5️⃣ 표 출력 및 저장
# ===============================================
table_res = extract_table(result_res, "거주용 빈집비율 베타회귀 결과")
table_nonres = extract_table(result_nonres, "비거주용 빈집비율 베타회귀 결과")

# 저장 경로 (Drive 연결 시 경로 바꾸면 됨)
table_res.to_csv("/content/거주용_빈집비율_베타회귀결과.csv", index=False, encoding="utf-8-sig")
table_nonres.to_csv("/content/비거주용_빈집비율_베타회귀결과.csv", index=False, encoding="utf-8-sig")

print("\n✅ CSV 파일이 /content/ 폴더에 저장되었습니다.")


🏠 거주용 빈집비율 회귀결과 (logit link)
Link function:logit
initial values are not assigned.
random initial values are assigned.

🏢 비거주용 빈집비율 회귀결과 (logit link)
Link function:logit
initial values are not assigned.
random initial values are assigned.

📊 거주용 빈집비율 베타회귀 결과


,변수명,계수(Beta)
0,m,0.0000
1,노후주택비율,-1.8854
2,인구밀도,-0.4135
3,순이동률,0.6993
4,1인가구수,1.6795



📊 비거주용 빈집비율 베타회귀 결과


,변수명,계수(Beta)
0,m,0.0000
1,경제활동인구,-1.2922
2,문화공간,0.5906
3,주택가격지수,-1.0470
4,교육만족도,-1.3091



✅ CSV 파일이 /content/ 폴더에 저장되었습니다.


In [ ]:
# ===============================================
# 1️⃣ 라이브러리
# ===============================================
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.special import logit

# ===============================================
# 2️⃣ 데이터 전처리
# ===============================================
df["거주용빈집비율"] = df["거주용빈집비율"].clip(0.001, 0.999)
df["비거주용빈집비율"] = df["비거주용빈집비율"].clip(0.001, 0.999)

features_res = ["문화공간", "인구밀도", "순이동률", "노후주택비율"]
features_nonres = ["경제활동인구", "문화공간", "인구밀도", "노인복지시설수"]

# 스케일 조정 (overflow 방지)
scaler = StandardScaler()
X_res_scaled = scaler.fit_transform(df[features_res])
X_nonres_scaled = scaler.fit_transform(df[features_nonres])

# 상수항 추가
X_res_scaled = sm.add_constant(X_res_scaled)
X_nonres_scaled = sm.add_constant(X_nonres_scaled)

# 종속변수 logit 변환 (베타회귀 근사)
y_res = logit(df["거주용빈집비율"])
y_nonres = logit(df["비거주용빈집비율"])

# ===============================================
# 3️⃣ 로짓 회귀 (GLM: Binomial family, logit link)
# ===============================================
model_res = sm.GLM(y_res, X_res_scaled, family=sm.families.Gaussian())
result_res = model_res.fit()

model_nonres = sm.GLM(y_nonres, X_nonres_scaled, family=sm.families.Gaussian())
result_nonres = model_nonres.fit()

# ===============================================
# 4️⃣ 결과 출력 (유의확률 포함)
# ===============================================
print("🏠 거주용 빈집비율 회귀 결과 (유의성 포함)")
print(result_res.summary())

print("\n🏢 비거주용 빈집비율 회귀 결과 (유의성 포함)")
print(result_nonres.summary())

# ===============================================
# 5️⃣ 결과 표 형태로 보기 쉽게 변환
# ===============================================
def make_result_table(result, features, title):
    df_result = pd.DataFrame({
        "변수명": ["절편"] + features,
        "계수(Beta)": result.params.round(4),
        "표준오차": result.bse.round(4),
        "z값": result.tvalues.round(3),
        "p값": result.pvalues.round(4)
    })
    print(f"\n📊 {title}")
    display(df_result)
    return df_result

table_res = make_result_table(result_res, features_res, "거주용 빈집비율 회귀결과")
table_nonres = make_result_table(result_nonres, features_nonres, "비거주용 빈집비율 회귀결과")

🏠 거주용 빈집비율 회귀 결과 (유의성 포함)
                 Generalized Linear Model Regression Results                  
Dep. Variable:                거주용빈집비율   No. Observations:                   24
Model:                            GLM   Df Residuals:                       19
Model Family:                Gaussian   Df Model:                            4
Link Function:               Identity   Scale:                         0.28696
Method:                          IRLS   Log-Likelihood:                -16.270
Date:                Thu, 23 Oct 2025   Deviance:                       5.4522
Time:                        13:01:37   Pearson chi2:                     5.45
No. Iterations:                     3   Pseudo R-squ. (CS):             0.3473
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5797     

,변수명,계수(Beta),표준오차,z값,p값
const,절편,-3.5797,0.1093,-32.738,0.0000
x1,문화공간,0.1070,0.1318,0.812,0.4170
x2,인구밀도,-0.2506,0.1344,-1.865,0.0622
x3,순이동률,0.0182,0.1141,0.159,0.8734
x4,노후주택비율,0.0906,0.1187,0.763,0.4454



📊 비거주용 빈집비율 회귀결과


,변수명,계수(Beta),표준오차,z값,p값
const,절편,-6.8447,0.0256,-267.380,0.0000
x1,경제활동인구,-0.0703,0.0361,-1.947,0.0515
x2,문화공간,0.1431,0.0307,4.659,0.0000
x3,인구밀도,0.0205,0.0303,0.678,0.4980
x4,노인복지시설수,-0.0261,0.0362,-0.721,0.4712


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 숫자형 변환
for col in df.columns:
    if col != "자치구":
        df[col] = pd.to_numeric(df[col], errors="coerce")

# 2. 종속변수: 빈집률 (% → 0~1)
df["전체빈집비율"] = df["전체빈집비율"] / 100
df["전체빈집비율"] = df["전체빈집비율"].clip(0.0001, 0.9999)

# 3. logit 변환
df["logit_빈집률"] = np.log(df["전체빈집비율"] / (1 - df["전체빈집비율"]))

# 4. 독립변수 정의
X_cols = ["노후주택비율","고령인구비율","인구밀도","순이동률",
          "경제활동인구","천명당_병상수","교육만족도","문화공간",
          "자동차등록수","주택가격지수","자살률","노인복지시설수","1인가구수"]

X = df[X_cols].fillna(df[X_cols].mean())
y = df["logit_빈집률"]

# 5. 회귀모형 적합
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

# 6. 결과 요약
print(model.summary())



In [ ]:
from patsy import dmatrices
import statsmodels.api as sm
import statsmodels.formula.api as smf

df["전체빈집비율"] = df["전체빈집비율"].clip(0.0001, 0.9999)

model = smf.glm(
    formula='Q("전체빈집비율") ~ Q("노후주택비율") + Q("고령인구비율") + Q("인구밀도") + Q("순이동률") + Q("경제활동인구")'
            ' + Q("천명당_병상수") + Q("교육만족도") + Q("문화공간") + Q("자동차등록수")'
            ' + Q("주택가격지수") + Q("자살률") + Q("노인복지시설수") + Q("1인가구수")',
    data=df,
    family=sm.families.Binomial(link=sm.families.links.logit())
)
result = model.fit()
print(result.summary())

In [ ]:
!apt-get -qq install fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

In [ ]:
!pip install esda libpysal splot geopandas shapely mapclassify

In [ ]:
# ===============================================
# 1️⃣ 라이브러리 불러오기
# ===============================================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import shap
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import geopandas as gpd
import esda
from libpysal.weights import Queen
from splot.esda import lisa_cluster

# ===============================================
# 2️⃣ 데이터 불러오기 및 분리
# ===============================================

# 예시: 이미 거주용, 비거주용 빈집율 컬럼이 있다고 가정
# (없다면 전체빈집수 대비 거주/비거주 빈집수로 직접 비율 생성 가능)
df["거주용빈집비율"] = df["거주용빈집비율"].clip(0.0001, 0.9999)
df["비거주용빈집비율"] = df["비거주용빈집비율"].clip(0.0001, 0.9999)

# 분석용 데이터 복사
df_res = df.copy()     # 거주용
df_nonres = df.copy()  # 비거주용


In [ ]:
# ===============================================
# 3️⃣ 기초통계 및 상관관계 시각화
# ===============================================
for target in ["거주용빈집비율", "비거주용빈집비율"]:
    print(f"\n[{target}] 기초통계")
    print(df[target].describe())
    sns.histplot(df[target], kde=True)
    plt.title(f"{target} 분포")
    plt.show()


sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("변수 상관관계 히트맵", fontsize=14)
plt.tight_layout()

# 🔸 이미지 저장
plt.savefig("/content/drive/MyDrive/corr_heatmap.png", dpi=300, bbox_inches="tight")
plt.close()

print("✅ corr_heatmap.png 저장 완료")


In [ ]:
import os

# ===============================================
# 📁 저장 경로 설정
# ===============================================
save_dir = "/content/drive/MyDrive/"
os.makedirs(save_dir, exist_ok=True)  # 폴더 없으면 자동 생성

# ===============================================
# 🔹 전체 상관계수 계산
# ===============================================
corr = df.corr(numeric_only=True).round(2)

# ===============================================
# 1️⃣ 거주용 빈집비율과 변수 간 상관관계 히트맵
# ===============================================
corr_res = corr[['거주용빈집비율']].sort_values(by='거주용빈집비율', ascending=False)

plt.figure(figsize=(4,6))
sns.heatmap(corr_res, annot=True, cmap="coolwarm", vmin=-1, vmax=1, cbar=True)
plt.title("거주용 빈집비율과 변수 간 상관관계", fontsize=13)
plt.tight_layout()

save_path_res = os.path.join(save_dir, "corr_residential_heatmap.png")
plt.savefig(save_path_res, dpi=300, bbox_inches="tight")
plt.close()

print(f"✅ 거주용 빈집비율 히트맵 저장 완료 → {save_path_res}")

# ===============================================
# 2️⃣ 비거주용 빈집비율과 변수 간 상관관계 히트맵
# ===============================================
corr_nonres = corr[['비거주용빈집비율']].sort_values(by='비거주용빈집비율', ascending=False)

plt.figure(figsize=(4,6))
sns.heatmap(corr_nonres, annot=True, cmap="coolwarm", vmin=-1, vmax=1, cbar=True)
plt.title("비거주용 빈집비율과 변수 간 상관관계", fontsize=13)
plt.tight_layout()

save_path_nonres = os.path.join(save_dir, "corr_nonresidential_heatmap.png")
plt.savefig(save_path_nonres, dpi=300, bbox_inches="tight")
plt.close()

print(f"✅ 비거주용 빈집비율 히트맵 저장 완료 → {save_path_nonres}")

In [ ]:
# ===============================================
# 🔹 상관계수 계산 + 제외 변수 설정
# ===============================================
corr = df.corr(numeric_only=True).round(2)
exclude_vars = ["전체빈집비율", "logit_빈집률", "거주용빈집비율", "비거주용빈집비율"]

# ===============================================
# 🔹 함수: 특정 변수와 상관 높은 상위 n개 변수 시각화
# ===============================================
def plot_top_corr_clean(df, target, top_n=7, color_palette="coolwarm"):
    corr_target = df.corr(numeric_only=True)[target].drop(target, errors="ignore")

    # 제외 변수 제거
    corr_target = corr_target.drop([v for v in exclude_vars if v in corr_target.index], errors="ignore")

    # 상관 절댓값 기준으로 정렬 후 상위 n개 선택
    corr_sorted = corr_target.reindex(corr_target.abs().sort_values(ascending=False).index)[:top_n]

    # 그래프 시각화
    plt.figure(figsize=(6,4))
    sns.barplot(x=corr_sorted.values, y=corr_sorted.index, palette=color_palette)
    plt.title(f"{target}과(와) 상관 높은 상위 {top_n} 변수", fontsize=13)
    plt.xlabel("상관계수")
    plt.tight_layout()

    filename = f"{target}_top{top_n}_corr_bar_clean.png"
    save_path = os.path.join(save_dir, filename)
    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close()

    print(f"✅ {filename} 저장 완료 → {save_path}")

# ===============================================
# 1️⃣ 거주용 빈집비율 상위 변수 저장
# ===============================================
plot_top_corr_clean(df, "거주용빈집비율", top_n=7, color_palette="Blues_r")

# ===============================================
# 2️⃣ 비거주용 빈집비율 상위 변수 저장
# ===============================================
plot_top_corr_clean(df, "비거주용빈집비율", top_n=7, color_palette="Oranges_r")

In [ ]:
# ===============================================
# 4️⃣ 다중공선성(VIF) 확인
# ===============================================
def calc_vif(data, cols):
    X = data[cols].dropna()
    X = sm.add_constant(X)
    vif = pd.DataFrame()
    vif["변수명"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif

predictors = ["노후주택비율","고령인구비율","인구밀도","순이동률","경제활동인구",
               "천명당_병상수","교육만족도","문화공간","자동차등록수",
               "주택가격지수","자살률","노인복지시설수","1인가구수"]

vif_df = calc_vif(df, predictors)
print(vif_df)


In [ ]:
# ===============================================
# 5️⃣ 베타회귀 (GLM 근사, logit 링크)
# ===============================================
# 🏠 거주용 모델
model_res = smf.glm(
    formula='Q("거주용빈집비율") ~ Q("노후주택비율") + Q("문화공간") + Q("인구밀도") + Q("순이동률") + Q("1인가구수")',
    data=df_res,
    family=sm.families.Binomial(link=sm.families.links.logit())
)
result_res = model_res.fit()
print(result_res.summary())

In [ ]:
# 🏢 비거주용 모델
model_nonres = smf.glm(
    formula='Q("비거주용빈집비율") ~ Q("경제활동인구") + Q("1인가구수") + Q("노인복지시설수") + Q("자동차등록수") + Q("인구밀도")',
    data=df_nonres,
    family=sm.families.Binomial(link=sm.families.links.logit())
)
result_nonres = model_nonres.fit()
print(result_nonres.summary())


In [ ]:
# ===============================================
# 6️⃣ 머신러닝(RandomForest/XGBoost + SHAP)
# ===============================================
features_res = ["노후주택비율","고령인구비율","인구밀도","순이동률","경제활동인구",
               "천명당_병상수","교육만족도","문화공간","자동차등록수",
               "주택가격지수","자살률","노인복지시설수","1인가구수"]
X_res = df_res[features_res]
y_res = df_res["거주용빈집비율"]

features_nonres = ["노후주택비율","고령인구비율","인구밀도","순이동률","경제활동인구",
               "천명당_병상수","교육만족도","문화공간","자동차등록수",
               "주택가격지수","자살률","노인복지시설수","1인가구수"]
X_nonres = df_nonres[features_nonres]
y_nonres = df_nonres["비거주용빈집비율"]

# 🔹 랜덤포레스트
rf_res = RandomForestRegressor(random_state=42)
rf_res.fit(X_res, y_res)

rf_nonres = RandomForestRegressor(random_state=42)
rf_nonres.fit(X_nonres, y_nonres)

# 🔹 SHAP
explainer_res = shap.Explainer(rf_res, X_res)
shap_values_res = explainer_res(X_res)

explainer_nonres = shap.Explainer(rf_nonres, X_nonres)
shap_values_nonres = explainer_nonres(X_nonres)

# SHAP 요약 플롯
print("🏠 거주용빈집율 SHAP 중요도")
shap.summary_plot(shap_values_res, X_res)

print("🏢 비거주용빈집율 SHAP 중요도")
shap.summary_plot(shap_values_nonres, X_nonres)

# ✅ 거주용 빈집비율
shap.summary_plot(shap_values_res, X_res, plot_type="bar",
                  color="royalblue", show=False)
plt.title("거주용 빈집비율 - 변수 중요도 (SHAP)", fontsize=14)
plt.tight_layout()
plt.show()

# ✅ 비거주용 빈집비율
shap.summary_plot(shap_values_nonres, X_nonres, plot_type="bar",
                  color="darkorange", show=False)
plt.title("비거주용 빈집비율 - 변수 중요도 (SHAP)", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
# ===============================================
# 🔸 XGBoost + SHAP
# ===============================================
from xgboost import XGBRegressor
import shap

# ⚙️ XGBoost 모델 학습 (거주용 / 비거주용 각각)
xgb_res = XGBRegressor(random_state=42)
xgb_res.fit(X_res, y_res)

xgb_nonres = XGBRegressor(random_state=42)
xgb_nonres.fit(X_nonres, y_nonres)

# ✅ SHAP 분석 (TreeExplainer 사용 권장)
explainer_xgb_res = shap.TreeExplainer(xgb_res)
shap_values_xgb_res = explainer_xgb_res.shap_values(X_res)

explainer_xgb_nonres = shap.TreeExplainer(xgb_nonres)
shap_values_xgb_nonres = explainer_xgb_nonres.shap_values(X_nonres)

# ===============================================
# 📊 시각화
# ===============================================
print("🏠 [XGBoost] 거주용 빈집비율 SHAP 중요도")
shap.summary_plot(shap_values_xgb_res, X_res)

print("🏢 [XGBoost] 비거주용 빈집비율 SHAP 중요도")
shap.summary_plot(shap_values_xgb_nonres, X_nonres)

# ✅ Bar Plot 형태로 요약 시각화
shap.summary_plot(shap_values_xgb_res, X_res, plot_type="bar",
                  color="seagreen", show=False)
plt.title("거주용 빈집비율 - 변수 중요도 (XGBoost + SHAP)", fontsize=14)
plt.tight_layout()
plt.show()

shap.summary_plot(shap_values_xgb_nonres, X_nonres, plot_type="bar",
                  color="purple", show=False)
plt.title("비거주용 빈집비율 - 변수 중요도 (XGBoost + SHAP)", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import shap

# ===============================================
# 📊 랜덤포레스트 + XGBoost SHAP 결과 저장
# ===============================================

# 분석 대상 리스트
models = [
    ("rf_res", shap_values_res, X_res, "royalblue", "거주용 빈집비율 (RandomForest)"),
    ("rf_nonres", shap_values_nonres, X_nonres, "darkorange", "비거주용 빈집비율 (RandomForest)"),
    ("xgb_res", shap_values_xgb_res, X_res, "seagreen", "거주용 빈집비율 (XGBoost)"),
    ("xgb_nonres", shap_values_xgb_nonres, X_nonres, "purple", "비거주용 빈집비율 (XGBoost)")
]

# 반복 저장
for name, shap_vals, X, color, title in models:
    # ------------------------
    # (1) Dot Plot 저장
    # ------------------------
    shap.summary_plot(shap_vals, X, show=False)
    plt.title(f"{title} - SHAP Dot Plot", fontsize=13)
    plt.tight_layout()
    plt.savefig(f"{name}_dot.png", dpi=300, bbox_inches="tight")
    plt.close()

    # ------------------------
    # (2) Bar Plot 저장
    # ------------------------
    shap.summary_plot(shap_vals, X, plot_type="bar", color=color, show=False)
    plt.title(f"{title} - SHAP 변수 중요도", fontsize=13)
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/{name}_bar.png", dpi=300, bbox_inches="tight")
    plt.close()

print("✅ 모든 SHAP 그래프 이미지 저장 완료!")
print("   (dot/bar plot 각각 .png 파일로 생성됨)")


In [ ]:
!pip install factor_analyzer

In [ ]:
# ===============================================
# 📊 요인분석 (Factor Analysis) + PCA 주성분분석
# ===============================================

from sklearn.preprocessing import StandardScaler
from factor_analyzer import FactorAnalyzer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# -----------------------------------------------
# 1️⃣ 분석 변수 선택
# -----------------------------------------------
# SHAP 분석에서 사용했던 주요 변수들 중심으로 선택
fa_vars = ["노후주택비율","고령인구비율","인구밀도","순이동률","경제활동인구",
               "천명당_병상수","교육만족도","문화공간","자동차등록수",
               "주택가격지수","자살률","노인복지시설수","1인가구수"]

df_fa = df[fa_vars].dropna()

# -----------------------------------------------
# 2️⃣ 표준화
# -----------------------------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_fa)

# -----------------------------------------------
# 3️⃣ 요인분석 (3요인, Varimax 회전)
# -----------------------------------------------
fa = FactorAnalyzer(n_factors=3, rotation='varimax')
fa.fit(X_scaled)

# 요인적재치 (Factor Loadings)
loadings = pd.DataFrame(fa.loadings_,
                        index=fa_vars,
                        columns=[f"요인{i+1}" for i in range(3)])
print("\n📌 [요인분석 결과] Varimax 회전 요인적재치")
print(loadings.round(3))

# -----------------------------------------------
# 4️⃣ Scree Plot (고유값으로 요인 수 판단)
# -----------------------------------------------
ev, v = fa.get_eigenvalues()
plt.figure(figsize=(6,4))
plt.plot(range(1, len(ev)+1), ev, marker='o')
plt.axhline(1, color='r', linestyle='--', label='고유값=1 기준')
plt.title("Scree Plot (요인 수 결정)")
plt.xlabel("요인 번호")
plt.ylabel("고유값")
plt.legend()
plt.show()

# -----------------------------------------------
# 5️⃣ Heatmap 시각화
# -----------------------------------------------
plt.figure(figsize=(8,6))
sns.heatmap(loadings, annot=True, cmap="coolwarm", center=0, fmt=".2f")
plt.title("요인적재치 (3요인, Varimax 회전)", fontsize=14)
plt.show()

# -----------------------------------------------
# 6️⃣ 자치구별 요인점수 계산 (LISA 등 공간분석에 활용 가능)
# -----------------------------------------------
factor_scores = fa.transform(X_scaled)
df_factors = pd.DataFrame(factor_scores, columns=["요인1","요인2","요인3"])
df_factors["자치구"] = df.loc[df_fa.index, "자치구"].values
print("\n✅ 자치구별 요인 점수 예시:")
print(df_factors.head())

## 주성분분석

In [ ]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame(pca_result, columns=["PC1","PC2"])
pca_df["자치구"] = df.loc[df_fa.index, "자치구"].values

print("\n📈 PCA 설명 분산 비율:", pca.explained_variance_ratio_)
print("누적 설명 비율:", round(sum(pca.explained_variance_ratio_)*100,2), "%")

# -----------------------------------------------
# 7️⃣ PCA 자치구 시각화
# -----------------------------------------------
plt.figure(figsize=(8,6))
sns.scatterplot(x='PC1', y='PC2', data=pca_df, s=100, color='skyblue', edgecolor='black')
for i, row in pca_df.iterrows():
    plt.text(row.PC1+0.05, row.PC2, row.자치구, fontsize=9)
plt.title("PCA 주성분 분석 (서울시 자치구별 요인 분포)", fontsize=13)
plt.xlabel(f"PC1 ({round(pca.explained_variance_ratio_[0]*100,1)}%)")
plt.ylabel(f"PC2 ({round(pca.explained_variance_ratio_[1]*100,1)}%)")
plt.grid(True)
plt.show()

# -----------------------------------------------
# 8️⃣ Biplot (변수 방향 시각화)
# -----------------------------------------------
loadings_pca = pd.DataFrame(pca.components_.T, columns=["PC1","PC2"], index=fa_vars)
plt.figure(figsize=(8,6))
plt.scatter(pca_df["PC1"], pca_df["PC2"], alpha=0.6, label="자치구")
for i, var in enumerate(fa_vars):
    plt.arrow(0, 0, loadings_pca.PC1[i]*2, loadings_pca.PC2[i]*2,
              color='red', alpha=0.6, head_width=0.03)
    plt.text(loadings_pca.PC1[i]*2.2, loadings_pca.PC2[i]*2.2, var, fontsize=8, color='darkred')
plt.title("PCA Biplot (변수-자치구 관계)", fontsize=13)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ===============================================
# 7️⃣ 공간 분석 (Moran’s I, LISA)
# ===============================================
import geopandas as gpd
import pandas as pd
from libpysal.weights import Queen
from esda.moran import Moran_Local
import matplotlib.pyplot as plt

# ===============================================
# 2️⃣ 데이터 불러오기 및 병합
# ===============================================
# shp 파일 (서울 자치구 경계)
gdf = gpd.read_file("/content/drive/MyDrive/sig.shp", encoding="euc-kr")

# 🔹 서울시만 필터링 (행정구역코드 11로 시작)
gdf = gdf[gdf['SIG_CD'].str.startswith("11")]

# ===============================================
# 3️⃣ 자치구 이름 정규화 (병합용)
# ===============================================
def normalize_name(x):
    if pd.isna(x):
        return ""
    x = str(x)
    x = x.replace("서울특별시", "").replace("서울시", "")
    x = x.replace(" ", "").replace("(", "").replace(")", "")
    if not x.endswith("구"):
        x = x + "구"
    return x

gdf["SIG_KOR_NM"] = gdf["SIG_KOR_NM"].apply(normalize_name)
df["자치구"] = df["자치구"].apply(normalize_name)

# 중복 컬럼 제거
df = df.loc[:, ~df.columns.duplicated()]

# 병합
mergedreal = gdf.merge(df, left_on="SIG_KOR_NM", right_on="자치구", how="inner")

# 병합 (이미 실행했을 테니까 아래 한 줄만 추가)
mergedreal = gpd.GeoDataFrame(mergedreal, geometry='geometry', crs=gdf.crs)

print(type(mergedreal))
print("geometry 포함 여부:", "geometry" in mergedreal.columns)

print("✅ 병합된 자치구 개수:", len(mergedreal))

# ===============================================
# 4️⃣ Moran’s I + LISA 분석 함수 정의
# ===============================================
def run_lisa_analysis(gdf, column_name, save_prefix):
    print(f"\n📊 {column_name} 공간자기상관 분석 시작...")

    # 1) Queen adjacency
    w = Queen.from_dataframe(gdf)
    w.transform = "r"

    # 2) Local Moran's I 계산
    x = gdf[column_name].values
    moran_loc = Moran_Local(x, w, permutations=999)

    # 3) 클러스터 라벨링
    labels = []
    for pv, q in zip(moran_loc.p_sim, moran_loc.q):
        if pv >= 0.05:
            labels.append("Not Significant")
        elif q == 1:
            labels.append("High-High")
        elif q == 2:
            labels.append("Low-High")
        elif q == 3:
            labels.append("Low-Low")
        elif q == 4:
            labels.append("High-Low")
        else:
            labels.append("Not Significant")

    gdf[f"{column_name}_cluster"] = labels
    gdf[f"{column_name}_I_local"] = moran_loc.Is
    gdf[f"{column_name}_p_value"] = moran_loc.p_sim
    gdf[f"{column_name}_z_score"] = moran_loc.z_sim

    # 4) 시각화
    cmap = {
        "High-High": "red",
        "Low-Low": "blue",
        "High-Low": "orange",
        "Low-High": "skyblue",
        "Not Significant": "lightgrey"
    }

    fig, ax = plt.subplots(1, 1, figsize=(9, 9))
    gdf.plot(ax=ax, color=gdf[f"{column_name}_cluster"].map(cmap),
             edgecolor="black", linewidth=0.8)

    ax.set_title(f"서울시 자치구 {column_name} LISA (Local Moran’s I)", fontsize=14)
    ax.axis("off")

    for lab, col in cmap.items():
        ax.scatter([], [], c=col, label=lab)
    ax.legend(loc="lower left", frameon=True)

    plt.savefig(f"{save_prefix}_LISA_map.png", dpi=300, bbox_inches="tight")
    plt.show()

    # 5) 결과 저장
    save_cols = ["자치구", column_name,
                 f"{column_name}_I_local", f"{column_name}_p_value",
                 f"{column_name}_z_score", f"{column_name}_cluster"]

    gdf[save_cols].to_csv(f"{save_prefix}_LISA_result.csv", index=False, encoding="utf-8-sig")
    gdf.to_file(f"{save_prefix}_LISA_result.geojson", driver="GeoJSON")

    print(f"✅ {column_name} LISA 결과 저장 완료 → {save_prefix}_LISA_result.*")

# ===============================================
# 5️⃣ 거주용 / 비거주용 각각 분석 실행
# ===============================================
run_lisa_analysis(mergedreal.copy(), "거주용빈집비율", "residential")
run_lisa_analysis(mergedreal.copy(), "비거주용빈집비율", "nonresidential")

## “요인1, 요인2, 요인3별 LISA(Local Moran’s I)”로“요인별 공간 군집 시각화 (예: 경제활동 집중 지역 vs 노후주택 집중 지역)”

In [ ]:
# ===============================================
# 📍 요인별 공간 자기상관 분석 (Moran’s I / LISA)
# ===============================================

import geopandas as gpd
from libpysal.weights import Queen
from esda.moran import Moran_Local
import matplotlib.pyplot as plt

# -----------------------------------------------
# 1️⃣ 공간 데이터 병합 (요인점수 + 경계데이터)
# -----------------------------------------------
# 혹시 공백, 괄호, ‘구’ 등의 표기가 다를 수 있으니 정규화 처리
df_factors["자치구"] = df_factors["자치구"].str.replace(" ", "").str.replace("구", "") + "구"
gdf["SIG_KOR_NM"] = gdf["SIG_KOR_NM"].str.replace(" ", "").str.replace("구", "") + "구"

# 병합 실행
gdf_factors = gdf.merge(df_factors, left_on="SIG_KOR_NM", right_on="자치구", how="inner")
print(f"✅ 병합된 자치구 수: {len(gdf_factors)} (정상: 25개 내외)")
# -----------------------------------------------
# 2️⃣ Queen contiguity weights 생성
# -----------------------------------------------
w = Queen.from_dataframe(gdf_factors)
w.transform = "r"

# -----------------------------------------------
# 3️⃣ LISA 분석 함수 정의
# -----------------------------------------------
def run_lisa_factor(gdf_input, column, save_prefix):
    print(f"\n📊 {column} LISA 분석 시작...")

    # Local Moran's I 계산
    moran_loc = Moran_Local(gdf_input[column], w, permutations=999)

    # LISA 클러스터 라벨링
    labels = []
    for pv, q in zip(moran_loc.p_sim, moran_loc.q):
        if pv >= 0.05:
            labels.append("Not Significant")
        elif q == 1:
            labels.append("High-High")
        elif q == 2:
            labels.append("Low-High")
        elif q == 3:
            labels.append("Low-Low")
        elif q == 4:
            labels.append("High-Low")
        else:
            labels.append("Not Significant")

    gdf_input[f"{column}_cluster"] = labels

    # -------------------------------------------
    # 시각화
    cmap = {
        "High-High": "red",
        "Low-Low": "blue",
        "High-Low": "orange",
        "Low-High": "skyblue",
        "Not Significant": "lightgrey"
    }

    fig, ax = plt.subplots(1, 1, figsize=(8,8))
    gdf_input.plot(ax=ax, color=gdf_input[f"{column}_cluster"].map(cmap),
                   edgecolor="black", linewidth=0.7)

    ax.set_title(f"서울시 자치구 {column} LISA (Local Moran’s I)", fontsize=14)
    ax.axis("off")

    # 범례 추가
    for lab, col in cmap.items():
        ax.scatter([], [], c=col, label=lab)
    ax.legend(loc="lower left", frameon=True)

    plt.show()

    # 결과 저장
    gdf_input[[column, f"{column}_cluster"]].to_csv(f"{save_prefix}_LISA_result.csv", index=False, encoding="utf-8-sig")
    gdf_input.to_file(f"{save_prefix}_LISA.geojson", driver="GeoJSON")

    print(f"✅ {column} LISA 분석 완료 → {save_prefix}_LISA_result.csv")

# -----------------------------------------------
# 4️⃣ 요인별 LISA 실행
# -----------------------------------------------
for i in range(1, 4):
    col = f"요인{i}"
    prefix = f"factor{i}"
    run_lisa_factor(gdf_factors.copy(), col, prefix)


In [ ]:
from libpysal.weights import Queen
from esda.moran import Moran_Local

# Queen contiguity weights 생성
w = Queen.from_dataframe(gdf_factors)
w.transform = "r"

def run_lisa_factor(gdf_input, column):
    moran_loc = Moran_Local(gdf_input[column], w, permutations=999)
    labels = []
    for pv, q in zip(moran_loc.p_sim, moran_loc.q):
        if pv >= 0.05:
            labels.append("Not Significant")
        elif q == 1:
            labels.append("High-High")
        elif q == 2:
            labels.append("Low-High")
        elif q == 3:
            labels.append("Low-Low")
        elif q == 4:
            labels.append("High-Low")
        else:
            labels.append("Not Significant")
    gdf_input[f"{column}_cluster"] = labels
    return gdf_input

# 요인별 LISA 실행
for i in range(1, 4):
    col = f"요인{i}"
    gdf_factors = run_lisa_factor(gdf_factors, col)

print("✅ 요인별 LISA 클러스터 컬럼 생성 완료")
print(gdf_factors.columns[-6:])  # 마지막 컬럼들 확인 (요인1_cluster 등 나와야 정상)


In [ ]:
# ===============================================
# 🎨 요인별 LISA 멀티패널 지도 (1x3 비교 시각화)
# ===============================================

import matplotlib.pyplot as plt

# -----------------------------------------------
# 1️⃣ 색상 매핑 (LISA 분류 공통)
# -----------------------------------------------
cmap_dict = {
    "High-High": "red",
    "Low-Low": "blue",
    "High-Low": "orange",
    "Low-High": "skyblue",
    "Not Significant": "lightgrey"
}

# -----------------------------------------------
# 2️⃣ 3개 지도 생성
# -----------------------------------------------
fig, axes = plt.subplots(1, 3, figsize=(18, 8))

for i, ax in enumerate(axes, start=1):
    col = f"요인{i}_cluster"
    if col not in gdf_factors.columns:
        print(f"⚠️ '{col}' 컬럼이 없음 — LISA 실행 후 다시 시도해 주세요.")
        continue

    gdf_factors.plot(ax=ax,
                     color=gdf_factors[col].map(cmap_dict),
                     edgecolor="black",
                     linewidth=0.6)

    ax.set_title(f"요인{i} LISA (Local Moran’s I)", fontsize=15)
    ax.axis("off")

# -----------------------------------------------
# 3️⃣ 공통 범례 추가
# -----------------------------------------------
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', label='High-High', markerfacecolor='red', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='Low-Low', markerfacecolor='blue', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='High-Low', markerfacecolor='orange', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='Low-High', markerfacecolor='skyblue', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='Not Significant', markerfacecolor='lightgrey', markersize=10),
]
fig.legend(handles=legend_elements, loc='lower center', ncol=5, frameon=True, fontsize=11)

plt.suptitle("서울시 자치구 요인별 LISA 공간패턴 비교", fontsize=18, y=0.95)
plt.tight_layout(rect=[0, 0.05, 1, 0.93])
plt.show()


In [ ]:
# =========================================================
# ✅ 누락된 거주용 / 비거주용 빈집비율 컬럼 병합
# =========================================================

# 원본 데이터(df 또는 merged)에서 자치구별 빈집비율 컬럼만 추출
missing_cols = ["거주용빈집비율", "비거주용빈집비율"]
existing_cols = [c for c in missing_cols if c in df.columns]

if len(existing_cols) > 0:
    vacancy_df = df[["자치구"] + existing_cols]
    print(f"병합할 컬럼: {existing_cols}")

    # 자치구 기준 병합
    gdf_factors = gdf_factors.merge(vacancy_df, on="자치구", how="left")
    print(f"✅ 병합 완료: {len(gdf_factors)}개 자치구, {len(gdf_factors.columns)}개 컬럼")
else:
    print("⚠️ df에 '거주용빈집비율' 또는 '비거주용빈집비율' 컬럼이 없습니다. 기존 df를 확인하세요.")

# 병합 후 컬럼 확인
print("\n📋 병합 후 gdf_factors 컬럼 목록:")
print(gdf_factors.columns.tolist())


In [ ]:
# ===============================================
# 🔍 ① 전역 Moran’s I 요약표 (Global Moran’s I)
# ===============================================
from esda.moran import Moran
import pandas as pd

moran_summary = []
cols = ["거주용빈집비율", "비거주용빈집비율", "요인1", "요인2", "요인3"]

for c in cols:
    moran = Moran(gdf_factors[c], w, permutations=999)
    moran_summary.append({
        "변수": c,
        "Moran’s I": round(moran.I, 3),
        "p-value": round(moran.p_sim, 4)
    })

moran_df = pd.DataFrame(moran_summary)
print("\n📊 전역 Moran’s I 요약표")
display(moran_df)

# 해석용 문장 출력
significant = moran_df[moran_df["p-value"] < 0.05]["변수"].tolist()
if significant:
    print(f"✅ 공간적 자기상관이 유의한 변수: {', '.join(significant)}")
else:
    print("❌ 유의한 공간적 자기상관이 발견되지 않았습니다.")


# ===============================================
# 🧩 ② SHAP – 요인 구조 연결 요약표
# ===============================================
# SHAP 상위 변수 추출
shap_mean_res = pd.DataFrame({
    "변수": X_res.columns,
    "거주용_SHAP": np.abs(shap_values_res.values).mean(axis=0)
}).sort_values("거주용_SHAP", ascending=False).head(7)

shap_mean_nonres = pd.DataFrame({
    "변수": X_nonres.columns,
    "비거주용_SHAP": np.abs(shap_values_nonres.values).mean(axis=0)
}).sort_values("비거주용_SHAP", ascending=False).head(7)

print("\n🏠 거주용 빈집 SHAP 상위 변수:")
display(shap_mean_res)
print("\n🏢 비거주용 빈집 SHAP 상위 변수:")
display(shap_mean_nonres)

# 요인별 변수 매핑 (요인적재치 기반)
mapping = loadings.idxmax(axis=1).reset_index()
mapping.columns = ["변수", "소속요인"]
print("\n📘 변수별 소속 요인 (요인적재치 기반)")
display(mapping)

# SHAP-요인 구조 통합
merged_shap_res = shap_mean_res.merge(mapping, on="변수", how="left")
merged_shap_nonres = shap_mean_nonres.merge(mapping, on="변수", how="left")

print("\n🏠 거주용 SHAP 변수의 요인 구조 연결:")
display(merged_shap_res)
print("\n🏢 비거주용 SHAP 변수의 요인 구조 연결:")
display(merged_shap_nonres)

# 자동 요약 문장
res_factors = merged_shap_res["소속요인"].value_counts().index.tolist()
nonres_factors = merged_shap_nonres["소속요인"].value_counts().index.tolist()
print(f"📈 거주용 빈집 주요 변수들은 주로 {', '.join(res_factors)}에 속함.")
print(f"📈 비거주용 빈집 주요 변수들은 주로 {', '.join(nonres_factors)}에 속함.")


# ===============================================
# 🎨 ③ LISA 멀티맵 요인명 자동 제목 버전
# ===============================================
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

factor_names = {
    1: "경제활동·인프라 요인",
    2: "사회·문화·생활만족 요인",
    3: "주거환경·고령화 요인"
}

cmap_dict = {
    "High-High": "red",
    "Low-Low": "blue",
    "High-Low": "orange",
    "Low-High": "skyblue",
    "Not Significant": "lightgrey"
}

fig, axes = plt.subplots(1, 3, figsize=(18, 8))

for i, ax in enumerate(axes, start=1):
    col = f"요인{i}_cluster"
    if col not in gdf_factors.columns:
        print(f"⚠️ '{col}' 컬럼이 없음 — LISA 실행 후 다시 시도해 주세요.")
        continue

    gdf_factors.plot(ax=ax, color=gdf_factors[col].map(cmap_dict),
                     edgecolor="black", linewidth=0.6)

    ax.set_title(f"{factor_names[i]} (LISA)", fontsize=14)
    ax.axis("off")

# 공통 범례
legend_elements = [
    Line2D([0], [0], marker='o', color='w', label='High-High', markerfacecolor='red', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='Low-Low', markerfacecolor='blue', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='High-Low', markerfacecolor='orange', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='Low-High', markerfacecolor='skyblue', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='Not Significant', markerfacecolor='lightgrey', markersize=10),
]
fig.legend(handles=legend_elements, loc='lower center', ncol=5, frameon=True, fontsize=11)

plt.suptitle("서울시 자치구 요인별 LISA 공간패턴 비교", fontsize=18, y=0.95)
plt.tight_layout(rect=[0, 0.05, 1, 0.93])
plt.show()


In [ ]:
# ===============================================
# 🔍 ① 전역 Moran’s I 요약표 (Global Moran’s I)
# ===============================================
from esda.moran import Moran
import pandas as pd

moran_summary = []
cols = ["거주용빈집비율", "비거주용빈집비율", "요인1", "요인2", "요인3"]

for c in cols:
    moran = Moran(gdf_factors[c], w, permutations=999)
    moran_summary.append({
        "변수": c,
        "Moran’s I": round(moran.I, 3),
        "p-value": round(moran.p_sim, 4)
    })

moran_df = pd.DataFrame(moran_summary)
print("\n📊 전역 Moran’s I 요약표")
display(moran_df)

# 해석용 문장 출력
significant = moran_df[moran_df["p-value"] < 0.05]["변수"].tolist()
if significant:
    print(f"✅ 공간적 자기상관이 유의한 변수: {', '.join(significant)}")
else:
    print("❌ 유의한 공간적 자기상관이 발견되지 않았습니다.")


# ===============================================
# 🧩 ② SHAP – 요인 구조 연결 요약표
# ===============================================
# SHAP 상위 변수 추출
shap_mean_res = pd.DataFrame({
    "변수": X_res.columns,
    "거주용_SHAP": np.abs(shap_values_res.values).mean(axis=0)
}).sort_values("거주용_SHAP", ascending=False).head(7)

shap_mean_nonres = pd.DataFrame({
    "변수": X_nonres.columns,
    "비거주용_SHAP": np.abs(shap_values_nonres.values).mean(axis=0)
}).sort_values("비거주용_SHAP", ascending=False).head(7)

print("\n🏠 거주용 빈집 SHAP 상위 변수:")
display(shap_mean_res)
print("\n🏢 비거주용 빈집 SHAP 상위 변수:")
display(shap_mean_nonres)

# 요인별 변수 매핑 (요인적재치 기반)
mapping = loadings.idxmax(axis=1).reset_index()
mapping.columns = ["변수", "소속요인"]
print("\n📘 변수별 소속 요인 (요인적재치 기반)")
display(mapping)

# SHAP-요인 구조 통합
merged_shap_res = shap_mean_res.merge(mapping, on="변수", how="left")
merged_shap_nonres = shap_mean_nonres.merge(mapping, on="변수", how="left")

print("\n🏠 거주용 SHAP 변수의 요인 구조 연결:")
display(merged_shap_res)
print("\n🏢 비거주용 SHAP 변수의 요인 구조 연결:")
display(merged_shap_nonres)

# 자동 요약 문장
res_factors = merged_shap_res["소속요인"].value_counts().index.tolist()
nonres_factors = merged_shap_nonres["소속요인"].value_counts().index.tolist()
print(f"📈 거주용 빈집 주요 변수들은 주로 {', '.join(res_factors)}에 속함.")
print(f"📈 비거주용 빈집 주요 변수들은 주로 {', '.join(nonres_factors)}에 속함.")


# ===============================================
# 🎨 ③ LISA 멀티맵 요인명 자동 제목 버전
# ===============================================
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

factor_names = {
    1: "경제활동·인프라 요인",
    2: "사회·문화·생활만족 요인",
    3: "주거환경·고령화 요인"
}

cmap_dict = {
    "High-High": "red",
    "Low-Low": "blue",
    "High-Low": "orange",
    "Low-High": "skyblue",
    "Not Significant": "lightgrey"
}

fig, axes = plt.subplots(1, 3, figsize=(18, 8))

for i, ax in enumerate(axes, start=1):
    col = f"요인{i}_cluster"
    if col not in gdf_factors.columns:
        print(f"⚠️ '{col}' 컬럼이 없음 — LISA 실행 후 다시 시도해 주세요.")
        continue

    gdf_factors.plot(ax=ax, color=gdf_factors[col].map(cmap_dict),
                     edgecolor="black", linewidth=0.6)

    ax.set_title(f"{factor_names[i]} (LISA)", fontsize=14)
    ax.axis("off")

# 공통 범례
legend_elements = [
    Line2D([0], [0], marker='o', color='w', label='High-High', markerfacecolor='red', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='Low-Low', markerfacecolor='blue', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='High-Low', markerfacecolor='orange', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='Low-High', markerfacecolor='skyblue', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='Not Significant', markerfacecolor='lightgrey', markersize=10),
]
fig.legend(handles=legend_elements, loc='lower center', ncol=5, frameon=True, fontsize=11)

plt.suptitle("서울시 자치구 요인별 LISA 공간패턴 비교", fontsize=18, y=0.95)
plt.tight_layout(rect=[0, 0.05, 1, 0.93])
plt.show()
